## 5-4. 소프트맥스 회귀 구현하기


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch. manual_seed(1)

In [2]:
x_train = [[1, 2, 1, 1],  #4개의 특성을 가지고 8개의 샘플 존재
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7],
          ]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]  #3개의 클래스(0,1,2) 존재#3개의 클래스 존재
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train) 

### 1. 소프트맥스 회귀 구현하기 (로우-레벨)

In [3]:
print(x_train.shape)  #8x4
print(y_train.shape)  #8x1 but 최종 사용할 레이블은 y_train에서 원-핫 인코딩한 결과여야 함. 8x3의 개수를 가져야 함.

torch.Size([8, 4])
torch.Size([8])


In [6]:
y_one_hot = torch.zeros(8,3)
y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
print(y_one_hot)
print(y_one_hot.shape)

tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])
torch.Size([8, 3])


In [7]:
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)  #W행렬의 크기는 4x3이어야 함
b = torch.zeros(1, requires_grad=True)

#optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)  #경사 하강법 사용

In [9]:
nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    #가설
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1)
    
    #비용 함수
    cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
    
    #cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    #100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{}  Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000  Cost: 1.098612
Epoch  100/1000  Cost: 0.761050
Epoch  200/1000  Cost: 0.689991
Epoch  300/1000  Cost: 0.643229
Epoch  400/1000  Cost: 0.604117
Epoch  500/1000  Cost: 0.568255
Epoch  600/1000  Cost: 0.533922
Epoch  700/1000  Cost: 0.500291
Epoch  800/1000  Cost: 0.466908
Epoch  900/1000  Cost: 0.433507
Epoch 1000/1000  Cost: 0.399962


### 2. 소프트맥스 회귀 구현하기 (하이-레벨)
F.cross_entropy()는 그 자체로 소프트맥스 함수 포함하고 있으므로 가설에서 소프트맥스 함수 사용할 필요 x

In [11]:
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)  #W행렬의 크기는 4x3이어야 함
b = torch.zeros(1, requires_grad=True)

#optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)  #경사 하강법 사용

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    #가설 x
    #hypothesis = F.softmax(x_train.matmul(W) + b, dim=1)
    
    #비용 함수
    z = x_train.matmul(W) + b
    cost = F.cross_entropy(z, y_train)
    
    #cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    #100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{}  Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000  Cost: 1.098612
Epoch  100/1000  Cost: 0.761050
Epoch  200/1000  Cost: 0.689991
Epoch  300/1000  Cost: 0.643229
Epoch  400/1000  Cost: 0.604117
Epoch  500/1000  Cost: 0.568255
Epoch  600/1000  Cost: 0.533922
Epoch  700/1000  Cost: 0.500291
Epoch  800/1000  Cost: 0.466908
Epoch  900/1000  Cost: 0.433507
Epoch 1000/1000  Cost: 0.399962


### 3. 소프트맥스 회귀 nn.Module로 구현하기
선형 회귀에서 구현에 사용했던 nn.Linear() 사용. output_dim이 1이 아닌 클래스의 개수여야 함

In [12]:
#모델을 선언 및 초기화. 4개의 특성을 가지고 3개의 클래스로 분류. input_dim=4, output_dim=3
model = nn.Linear(4,3)

In [13]:
#optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)  #경사 하강법 사용

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    #비용 함수
    z = x_train.matmul(W) + b
    cost = F.cross_entropy(z, y_train)
    
    #cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    #100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{}  Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000  Cost: 0.385264
Epoch  100/1000  Cost: 0.385264
Epoch  200/1000  Cost: 0.385264
Epoch  300/1000  Cost: 0.385264
Epoch  400/1000  Cost: 0.385264
Epoch  500/1000  Cost: 0.385264
Epoch  600/1000  Cost: 0.385264
Epoch  700/1000  Cost: 0.385264
Epoch  800/1000  Cost: 0.385264
Epoch  900/1000  Cost: 0.385264
Epoch 1000/1000  Cost: 0.385264


### 4. 소프트맥스 회귀 클래스로 구현하기

In [15]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3) #Output이 3
        
    def forward(self, x):
        return self.linear(x)
    
model = SoftmaxClassifierModel()

In [16]:
#optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)  #경사 하강법 사용

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    #H(x) 계산
    prediction = model(x_train)
    
    #비용 함수
    cost = F.cross_entropy(prediction, y_train)
    
    #cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    #100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{}  Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000  Cost: 2.637636
Epoch  100/1000  Cost: 0.647903
Epoch  200/1000  Cost: 0.564643
Epoch  300/1000  Cost: 0.511043
Epoch  400/1000  Cost: 0.467249
Epoch  500/1000  Cost: 0.428281
Epoch  600/1000  Cost: 0.391924
Epoch  700/1000  Cost: 0.356742
Epoch  800/1000  Cost: 0.321577
Epoch  900/1000  Cost: 0.285617
Epoch 1000/1000  Cost: 0.250818
